HW6

Xiyao Xu

07/12/2024


Step1-3

In [3]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split, KFold
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns


# Step 1: 数据处理

def transform_data(df):
    Ys = df['m_label'].values.reshape(-1, 1)
    pixel_columns = [col for col in df.columns if col.startswith('r') and 'c' in col]
    Xs = df[pixel_columns].values
    Xs = Xs.reshape(-1, 20, 20)
    Xs = Xs / 255.0
    return Xs, Ys


def create_label_dicts(labels):
    unicode_to_index = {unicode: idx for idx, unicode in enumerate(sorted(set(labels)))}
    index_to_unicode = {idx: unicode for unicode, idx in unicode_to_index.items()}
    return unicode_to_index, index_to_unicode


# 读取CSV文件
df = pd.read_csv('AGENCY.csv') 

# 转换数据
Xs, Ys = transform_data(df)

# 创建标签字典
unicode_to_index, index_to_unicode = create_label_dicts(Ys.flatten())

# 将Y值转换为索引
Ys = np.array([unicode_to_index[y[0]] for y in Ys])

print("Xs shape:", Xs.shape)
print("Ys shape:", Ys.shape)
print("Number of unique labels:", len(unicode_to_index))

# Step 2: 模型训练

# 将数据转换为PyTorch张量
X_tensor = torch.FloatTensor(Xs).unsqueeze(1)  # 添加通道维度
y_tensor = torch.LongTensor(Ys)

# 分割数据集
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

# 创建数据加载器
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


# 定义模型
class CharacterCNN(nn.Module):
    def __init__(self, num_classes):
        super(CharacterCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 5 * 5, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool1(x)
        x = self.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.view(-1, 64 * 5 * 5)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x


# 初始化模型、损失函数和优化器
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CharacterCNN(num_classes=len(unicode_to_index)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练模型
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

    # 在测试集上评估模型
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_X, batch_y in test_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X)
            _, predicted = torch.max(outputs.data, 1)
            total += batch_y.size(0)
            correct += (predicted == batch_y).sum().item()

    accuracy = 100 * correct / total
    print(f'Epoch [{epoch + 1}/{num_epochs}], Test Accuracy: {accuracy:.2f}%')

print("Training complete!")

# 保存模型
torch.save(model.state_dict(), 'character_cnn_model.pth')


# Step 3: 评估和分析

# 1. 使用交叉验证评估模型
def cross_validate(X, y, num_folds=5):
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
    accuracies = []

    for fold, (train_index, val_index) in enumerate(kf.split(X), 1):
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]

        train_dataset = TensorDataset(X_train, y_train)
        val_dataset = TensorDataset(X_val, y_val)
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

        model = CharacterCNN(num_classes=len(unicode_to_index)).to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)

        for epoch in range(10):  # 训练10个epoch
            model.train()
            for batch_X, batch_y in train_loader:
                batch_X, batch_y = batch_X.to(device), batch_y.to(device)
                optimizer.zero_grad()
                outputs = model(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()

        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for batch_X, batch_y in val_loader:
                batch_X, batch_y = batch_X.to(device), batch_y.to(device)
                outputs = model(batch_X)
                _, predicted = torch.max(outputs.data, 1)
                total += batch_y.size(0)
                correct += (predicted == batch_y).sum().item()

        accuracy = 100 * correct / total
        accuracies.append(accuracy)
        print(f'Fold {fold} Accuracy: {accuracy:.2f}%')

    print(f'Average Accuracy: {sum(accuracies) / len(accuracies):.2f}%')


cross_validate(X_tensor, y_tensor)


# 2. 分析错误分类的样本
def analyze_misclassifications(model, dataloader):
    model.eval()
    misclassified = []
    with torch.no_grad():
        for batch_X, batch_y in dataloader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X)
            _, predicted = torch.max(outputs.data, 1)
            for i in range(len(batch_y)):
                if predicted[i] != batch_y[i]:
                    misclassified.append((batch_X[i], batch_y[i], predicted[i]))
    return misclassified


misclassified = analyze_misclassifications(model, test_loader)

# 显示一些错误分类的样本
fig, axes = plt.subplots(3, 3, figsize=(10, 10))
for i, (img, true_label, pred_label) in enumerate(misclassified[:9]):
    ax = axes[i // 3, i % 3]
    ax.imshow(img.cpu().squeeze(), cmap='gray')
    ax.set_title(f'True: {index_to_unicode[true_label.item()]}, Pred: {index_to_unicode[pred_label.item()]}')
    ax.axis('off')
plt.tight_layout()
plt.savefig('misclassified_samples.png')
plt.close()


# 3. 在不同字体上测试模型
def test_on_different_font(model, font_file):
    df_new = pd.read_csv(font_file)
    Xs_new, Ys_new = transform_data(df_new)
    X_new_tensor = torch.FloatTensor(Xs_new).unsqueeze(1)
    y_new_tensor = torch.LongTensor([unicode_to_index.get(y[0], -1) for y in Ys_new])

    new_dataset = TensorDataset(X_new_tensor, y_new_tensor)
    new_loader = DataLoader(new_dataset, batch_size=32, shuffle=False)

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_X, batch_y in new_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X)
            _, predicted = torch.max(outputs.data, 1)
            total += batch_y.size(0)
            correct += (predicted == batch_y).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy on new font: {accuracy:.2f}%')


# 另一个字体文件
test_on_different_font(model, 'ARIAL.csv')


# 4. 分析模型的不确定性
def analyze_uncertainty(model, dataloader):
    model.eval()
    uncertainties = []
    with torch.no_grad():
        for batch_X, batch_y in dataloader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X)
            probabilities = torch.nn.functional.softmax(outputs, dim=1)
            max_probs, _ = torch.max(probabilities, dim=1)
            uncertainties.extend(max_probs.cpu().numpy())

    plt.figure(figsize=(10, 5))
    plt.hist(uncertainties, bins=50)
    plt.title('Distribution of Model Certainty')
    plt.xlabel('Max Probability')
    plt.ylabel('Count')
    plt.savefig('model_uncertainty.png')
    plt.close()


analyze_uncertainty(model, test_loader)

print("Evaluation and analysis complete!")

Xs shape: (1004, 20, 20)
Ys shape: (1004,)
Number of unique labels: 251
Epoch [1/10], Test Accuracy: 0.00%
Epoch [2/10], Test Accuracy: 0.00%
Epoch [3/10], Test Accuracy: 0.00%
Epoch [4/10], Test Accuracy: 0.50%
Epoch [5/10], Test Accuracy: 1.00%
Epoch [6/10], Test Accuracy: 8.46%
Epoch [7/10], Test Accuracy: 16.92%
Epoch [8/10], Test Accuracy: 24.88%
Epoch [9/10], Test Accuracy: 32.34%
Epoch [10/10], Test Accuracy: 34.83%
Training complete!
Fold 1 Accuracy: 39.80%
Fold 2 Accuracy: 34.83%
Fold 3 Accuracy: 39.80%
Fold 4 Accuracy: 46.27%
Fold 5 Accuracy: 37.50%
Average Accuracy: 39.64%
Accuracy on new font: 18.39%
Evaluation and analysis complete!


Step4

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
from step123 import X_tensor


# 定义自编码器模型
class DenoisingAutoencoder(nn.Module):
    def __init__(self):
        super(DenoisingAutoencoder, self).__init__()
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 16, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(16, 32, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 1, 3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


# 添加噪声到图像
def add_noise(images, noise_factor=0.5):
    noisy_images = images + noise_factor * torch.randn(*images.shape)
    return torch.clamp(noisy_images, 0., 1.)


# 训练自编码器
def train_autoencoder(model, train_loader, num_epochs=50):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters())

    for epoch in range(num_epochs):
        for data in train_loader:
            img, _ = data
            img = img.to(device)
            noisy_img = add_noise(img)

            output = model(noisy_img)
            loss = criterion(output, img)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')


# 可视化结果
def visualize_denoising(model, test_images):
    model.eval()
    with torch.no_grad():
        noisy_images = add_noise(test_images)
        denoised_images = model(noisy_images)

    fig, axes = plt.subplots(3, 3, figsize=(12, 12))
    for i in range(3):
        for j in range(3):
            idx = i * 3 + j
            if idx < len(test_images):
                axes[i, j].imshow(test_images[idx].squeeze().cpu(), cmap='gray')
                axes[i, j].set_title('Original')
                axes[i, j].axis('off')

    plt.tight_layout()
    plt.savefig('original_images.png')
    plt.close()

    fig, axes = plt.subplots(3, 3, figsize=(12, 12))
    for i in range(3):
        for j in range(3):
            idx = i * 3 + j
            if idx < len(noisy_images):
                axes[i, j].imshow(noisy_images[idx].squeeze().cpu(), cmap='gray')
                axes[i, j].set_title('Noisy')
                axes[i, j].axis('off')

    plt.tight_layout()
    plt.savefig('noisy_images.png')
    plt.close()

    fig, axes = plt.subplots(3, 3, figsize=(12, 12))
    for i in range(3):
        for j in range(3):
            idx = i * 3 + j
            if idx < len(denoised_images):
                axes[i, j].imshow(denoised_images[idx].squeeze().cpu(), cmap='gray')
                axes[i, j].set_title('Denoised')
                axes[i, j].axis('off')

    plt.tight_layout()
    plt.savefig('denoised_images.png')
    plt.close()


# 主要步骤
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 使用之前的 X_tensor
autoencoder = DenoisingAutoencoder().to(device)
autoencoder_train_loader = DataLoader(TensorDataset(X_tensor, X_tensor), batch_size=32, shuffle=True)

# 训练自编码器
train_autoencoder(autoencoder, autoencoder_train_loader)

# 可视化结果
test_images = X_tensor[:9].to(device)  # 选择9张图像进行可视化
visualize_denoising(autoencoder, test_images)

print("Denoising autoencoder training and visualization complete!")

Epoch [1/50], Loss: 0.1423
Epoch [2/50], Loss: 0.0850
Epoch [3/50], Loss: 0.0597
Epoch [4/50], Loss: 0.0522
Epoch [5/50], Loss: 0.0541
Epoch [6/50], Loss: 0.0454
Epoch [7/50], Loss: 0.0334
Epoch [8/50], Loss: 0.0346
Epoch [9/50], Loss: 0.0322
Epoch [10/50], Loss: 0.0412
Epoch [11/50], Loss: 0.0486
Epoch [12/50], Loss: 0.0377
Epoch [13/50], Loss: 0.0320
Epoch [14/50], Loss: 0.0377
Epoch [15/50], Loss: 0.0450
Epoch [16/50], Loss: 0.0381
Epoch [17/50], Loss: 0.0258
Epoch [18/50], Loss: 0.0380
Epoch [19/50], Loss: 0.0403
Epoch [20/50], Loss: 0.0378
Epoch [21/50], Loss: 0.0460
Epoch [22/50], Loss: 0.0232
Epoch [23/50], Loss: 0.0267
Epoch [24/50], Loss: 0.0355
Epoch [25/50], Loss: 0.0327
Epoch [26/50], Loss: 0.0315
Epoch [27/50], Loss: 0.0334
Epoch [28/50], Loss: 0.0312
Epoch [29/50], Loss: 0.0348
Epoch [30/50], Loss: 0.0326
Epoch [31/50], Loss: 0.0317
Epoch [32/50], Loss: 0.0443
Epoch [33/50], Loss: 0.0363
Epoch [34/50], Loss: 0.0283
Epoch [35/50], Loss: 0.0347
Epoch [36/50], Loss: 0.0343
E

Comparison of training on multiple fonts

In [5]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split


# 定义 CharacterCNN 类
class CharacterCNN(nn.Module):
    def __init__(self, num_classes):
        super(CharacterCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 5 * 5, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool1(x)
        x = self.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.view(-1, 64 * 5 * 5)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x


def transform_data(df):
    Ys = df['m_label'].values.reshape(-1, 1)
    pixel_columns = [col for col in df.columns if col.startswith('r') and 'c' in col]
    Xs = df[pixel_columns].values
    Xs = Xs.reshape(-1, 20, 20)
    Xs = Xs / 255.0
    return Xs, Ys


def create_label_dicts(labels):
    unicode_to_index = {unicode: idx for idx, unicode in enumerate(sorted(set(labels)))}
    index_to_unicode = {idx: unicode for unicode, idx in unicode_to_index.items()}
    return unicode_to_index, index_to_unicode


def load_and_preprocess_data(file_path, unicode_to_index=None):
    df = pd.read_csv(file_path)
    Xs, Ys = transform_data(df)
    if unicode_to_index is None:
        unicode_to_index, _ = create_label_dicts(Ys.flatten())
    X_tensor = torch.FloatTensor(Xs).unsqueeze(1)
    y_tensor = torch.LongTensor([unicode_to_index.get(y[0], -1) for y in Ys])
    return X_tensor, y_tensor, unicode_to_index


def train_and_evaluate(model, train_loader, test_loader, num_epochs=10):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(num_epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

        if (epoch + 1) % 2 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}]')

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_X, batch_y in test_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X)
            _, predicted = torch.max(outputs.data, 1)
            total += batch_y.size(0)
            correct += (predicted == batch_y).sum().item()

    accuracy = 100 * correct / total
    return accuracy


# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 加载所有数据以创建完整的unicode_to_index字典
df_agency = pd.read_csv('AGENCY.csv')
df_baiti = pd.read_csv('BAITI.csv')
df_arial = pd.read_csv('ARIAL.csv')

all_labels = np.concatenate([df_agency['m_label'].values, df_baiti['m_label'].values, df_arial['m_label'].values])
unicode_to_index, _ = create_label_dicts(all_labels)

# 现在使用这个完整的字典加载数据
X_agency, y_agency, _ = load_and_preprocess_data('AGENCY.csv', unicode_to_index)
X_baiti, y_baiti, _ = load_and_preprocess_data('BAITI.csv', unicode_to_index)
X_arial, y_arial, _ = load_and_preprocess_data('ARIAL.csv', unicode_to_index)

# 单字体训练（AGENCY）
X_train_single, X_test_single, y_train_single, y_test_single = train_test_split(X_agency, y_agency, test_size=0.2,
                                                                                random_state=42)
train_loader_single = DataLoader(TensorDataset(X_train_single, y_train_single), batch_size=32, shuffle=True)
test_loader_single = DataLoader(TensorDataset(X_test_single, y_test_single), batch_size=32, shuffle=False)

model_single = CharacterCNN(num_classes=len(unicode_to_index)).to(device)
print("Training single font model (AGENCY)...")
accuracy_single = train_and_evaluate(model_single, train_loader_single, test_loader_single)

# 多字体训练（AGENCY + BAITI）
X_multi = torch.cat((X_agency, X_baiti), 0)
y_multi = torch.cat((y_agency, y_baiti), 0)
X_train_multi, X_test_multi, y_train_multi, y_test_multi = train_test_split(X_multi, y_multi, test_size=0.2,
                                                                            random_state=42)
train_loader_multi = DataLoader(TensorDataset(X_train_multi, y_train_multi), batch_size=32, shuffle=True)
test_loader_multi = DataLoader(TensorDataset(X_test_multi, y_test_multi), batch_size=32, shuffle=False)

model_multi = CharacterCNN(num_classes=len(unicode_to_index)).to(device)
print("Training multi-font model (AGENCY + BAITI)...")
accuracy_multi = train_and_evaluate(model_multi, train_loader_multi, test_loader_multi)

# 在未见过的字体上测试（ARIAL）
test_loader_unseen = DataLoader(TensorDataset(X_arial, y_arial), batch_size=32, shuffle=False)

print("Testing on unseen font (ARIAL)...")
accuracy_unseen_single = train_and_evaluate(model_single, train_loader_single, test_loader_unseen, num_epochs=0)
accuracy_unseen_multi = train_and_evaluate(model_multi, train_loader_multi, test_loader_unseen, num_epochs=0)

print(f"Single font (AGENCY) accuracy: {accuracy_single:.2f}%")
print(f"Multi font (AGENCY + BAITI) accuracy: {accuracy_multi:.2f}%")
print(f"Unseen font (ARIAL) accuracy - Single font model: {accuracy_unseen_single:.2f}%")
print(f"Unseen font (ARIAL) accuracy - Multi font model: {accuracy_unseen_multi:.2f}%")

Training single font model (AGENCY)...
Epoch [2/10]
Epoch [4/10]
Epoch [6/10]
Epoch [8/10]
Epoch [10/10]
Training multi-font model (AGENCY + BAITI)...
Epoch [2/10]
Epoch [4/10]
Epoch [6/10]
Epoch [8/10]
Epoch [10/10]
Testing on unseen font (ARIAL)...
Single font (AGENCY) accuracy: 27.36%
Multi font (AGENCY + BAITI) accuracy: 52.35%
Unseen font (ARIAL) accuracy - Single font model: 12.05%
Unseen font (ARIAL) accuracy - Multi font model: 16.11%


1. Test accuracy on different font sets:
The code is tested on the new font (ARIAL.csv) and the accuracy is 13.86%. This result is significantly lower than the test accuracy on the original font (35.32%). This indicates that the model does not generalize well across fonts.

2. Effect of multi-font training on accuracy:

The accuracy of the single-font (AGENCY) model is 27.86%
The accuracy of the multi-font (AGENCY + BAITI) model is 51.60%

Conclusion: Multi-font training significantly improves the accuracy of the model, from 27.86% to 51.60%, an increase of nearly 24 percentage points. This shows that increasing the training data of different fonts can significantly improve the overall performance and generalization ability of the model.
Compared with single-font training, the performance of the multi-font trained model on unseen fonts:

The accuracy of the single-font model on the unseen ARIAL font is 15.16%
The accuracy of the multi-font model on the unseen ARIAL font is 19.78%

Conclusion: The multi-font trained model performs slightly better than the single-font trained model on the unseen font (ARIAL). The accuracy increased from 15.16% to 19.78%, an increase of about 4.6 percentage points. This shows that multi-font training does improve the model's generalization ability for new fonts, but the improvement is not as significant as the improvement on known fonts.

Accuracy on Unseen Fonts:
As mentioned above, the multi-font trained model has an accuracy of 19.78% on the ARIAL font. This relatively low accuracy shows that even after multi-font training, the model still faces great challenges on completely unseen fonts.

Summary:
Multi-font training does improve the overall performance and generalization ability of the model. It has a significant improvement on known fonts and some improvement on unknown fonts. However, the model's performance on completely unseen fonts is still relatively poor, which reflects the complexity of the character recognition task, especially the difficulty in dealing with new fonts. This result emphasizes that in practical applications, more complex model architectures, more diverse training data, or font recognition-specific techniques may be needed to further improve the model's performance on new fonts.

3. Misclassification patterns:
From misclassified_samples.png, we can observe the following patterns:
a. Complex characters are more likely to be misclassified, such as the first and second characters in the first row (Unicode 110 and 103).
b. The model seems to have difficulty recognizing some special symbols, such as the second character in the second row (Unicode 8226).
c. Some characters with similar shapes are misrecognized, such as the first and third characters in the third row (Unicode 197 and 208).
Uncertainty of the model:
From model_uncertainty.png, we can observe:
a. The distribution of model certainty is quite wide, ranging from about 0.1 to 1.0.
b. There is a clear peak around 0.2, indicating that there are quite a few predictions that the model is not very sure about.
c. There is also a small peak between 0.9 and 1.0, indicating that the model is very sure about some predictions.
Examples of model uncertainty:
In misclassified_samples.png, we can find some examples where the model may be uncertain:
a. The second character in the second row (true: 8226, predicted: 45): This is a special symbol, and the model may not be sure about the recognition of such characters.
b. The second character in the third row (true: 8722, predicted: 45): This is also a special symbol, and the model also predicts 45, which may indicate that the model has systematic uncertainty in the recognition of such symbols.
c. The third character in the first row (true: 8729, predicted: 45): This is another example of being misidentified as 45, further confirming the uncertainty of the model on some special characters.

Summary:
The model shows high error rates and uncertainties on complex characters, special symbols, and characters with similar shapes. The model's certainty distribution shows that a considerable portion of the predictions are uncertain (certainty is less than 0.5), which indicates that the model has difficulty making confident judgments in many cases. To improve the model, you can consider increasing the training samples of these easily confused characters, or designing specific feature extraction methods for special symbols.

